In [1]:
import networkx as nx
import random

In [85]:
def independent_cascade_model(G, seeds, steps=0):
    
    # On verifie que les seeds sont dans G
    for s in seeds:
        if s not in G.nodes():
            raise Exception("seed", s, "is not in graph")

    # init activation probabilities
    for e in G.edges():
        if 'act_prob' not in G[e[0]][e[1]]:
            G[e[0]][e[1]]['act_prob'] = 0.1
        elif G[e[0]][e[1]]['act_prob'] > 1:
            raise Exception("link probability > 1")

    #diffusion
    A = seeds[:]  
    if steps <= 0:
    # perform diffusion until no more nodes can be activated
        print "performing diffusion on all nodes"
        return diffuse_all(G, A)
    
    # perform diffusion for at most "steps" rounds
    return diffuse_k_rounds(G, A, steps)

In [125]:
#cas step =< 0
def diffuse_all(G, A):
    tried_links = set()
    layer_i_nodes = [ ]
    #initialisation des seeds
    layer_i_nodes.append([i for i in A])
    
    while True:
        len_old = len(A)
        #performing on step diffusion
        (A, activated_nodes_of_this_round, current_tried_links) = diffuse_one_round(G, A, tried_links)
        layer_i_nodes.append(activated_nodes_of_this_round)
        tried_links = tried_links.union(current_tried_links)
        if len(A) == G.number_of_nodes():
            break
    return layer_i_nodes

In [126]:
def diffuse_one_round(G, A, tried_links):
    #initialisation
    activated_nodes_round = set()
    current_tried_link = set()
    for s in A:
        for nb in G.successors(s):
            if nb in A or (s, nb) in current_tried_link:
                continue
            if _prop_success(G, s, nb):
                activated_nodes_round.add(nb)
            current_tried_link.add((s, nb))
    activated_nodes_round = list(activated_nodes_round)
    A.extend(activated_nodes_round)
    return A, activated_nodes_round, current_tried_link

In [127]:
def diffuse_k_rounds(G, A, steps):
    tried_link = set()
    layer_i_nodes = [ ]
    layer_i_nodes.append([i for i in A])
    while steps > 0 and len(A) < len(G):
        len_old = len(A)
        (A, activated_nodes_of_this_round, current_tried_link) = diffuse_one_round(G, A, tried_link)
        layer_i_nodes.append(activated_nodes_of_this_round)
        tried_link = tried_link.union(current_tried_link)
        if len(A) == len_old:
            break
        steps -= 1
    return layer_i_nodes

In [128]:
def _prop_success(G, src, dest):
    ''' reussite propagation'''
    return random.random() <= G[src][dest]['act_prob']

In [131]:
DG = nx.DiGraph()
DG.add_edges_from([(1,2), (1,3), (1,5), (2,1), (3,2), (4,2), (4,3),(4,6), (5,3), (5,4), (5,6), (6,4), (6,5)], act_prob=0.7)

In [132]:
H = independent_cascade_model(DG,[6])
H

performing diffusion on all nodes


[[6], [5], [3, 4], [2], [1]]

# Apprentissage

In [74]:
import numpy as np

#initialisation
nbr_node = DG.number_of_nodes()
Prob = np.random.randn(nbr_node,1)

In [ ]:
Prob_ 